In [ ]:
import cv2
import glob
import numpy as np
import math
import os

In [ ]:
#18gig zip file! may take a while
!wget --no-check-certificate \
    http://images.cocodataset.org/zips/train2017.zip \
    -O ./train2017.zip


In [ ]:

import zipfile

local_zip = './train2017.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/train2017')
zip_ref.close()

In [ ]:
# this selects the first 1975 images, change the regex if you want more
img_paths = glob.glob('./train2017/00000000[0-9]*.jpg')
training_imgs_num = len(img_paths)
print(f'loaded a total of {training_imgs_num} imgs')

In [ ]:
print(f'Cleaning and saving {training_imgs_num} imgs to ./cleaned-data')
os.mkdir( './cleaned-data/') if not os.path.exists('./cleaned-data/') else None

for img_path in img_paths:
    img = cv2.imread(img_path)
    y,x,_ = img.shape #(y, x) not (x, y)
    
    margin = abs(y-x)/2
    if x > y:        
        # Image is tall
        img = img[:,int(math.floor(margin)):int(math.floor(x-margin))]
    elif y > x:
        # Image is wide
        img = img[int(math.floor(margin)):int(math.floor(y-margin)),:]

    if (img.shape[0] != img.shape[1]):
        print('Dim mismatch')
        
    img = cv2.resize(img, (128,128))
    cv2.imwrite(os.path.join(os.curdir, 'cleaned-data', img_path.split('\\')[-1]), img)

In [148]:
import json

with open('./annotations_trainval2017/annotations/captions_train2017.json') as annot_file:
    captions_df = pd.read_json(annot_file, typ='series')
    annot_df = pd.DataFrame(data=captions_df['annotations'])
    annot_df = annot_df.astype({'image_id': 'int32'})
    annot_df.sort_values(by=['image_id'], axis=0, inplace=True)
    annot_df = annot_df.groupby('image_id')['image_id', 'caption'].agg('|'.join)
    

    with open('./annotations_trainval2017/annotations/instances_train2017.json') as instance_file:
        instances_df = pd.read_json(instance_file, typ='series')
#         instances_df.annotations
        

In [174]:
summed = annot_df.groupby('image_id')['image_id', 'caption'].agg('|'.join)

In [182]:
summed.to_csv('./all-captions.csv', index=True, header=False)

In [120]:
labels = [x['category_id'] for x in df.annotations if x['image_id'] == 558840]

[58, 44, 44, 47, 1, 44, 50, 1, 44, 67, 50]